In [1]:
pip install langchain langchain_community langchain_chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "false"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [3]:
from langchain_ollama import ChatOllama

model_name="deepseek-r1:8b"
llm = ChatOllama(model=model_name,
                            base_url="http://10.168.6.34:11434",temperature = 0,mirostat_tau=2.0,top_k=10,top_p=0.5
                            )

In [4]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
len(docs[0].page_content)

43130

In [6]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [ ]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 使用关键字参数初始化 OllamaEmbeddings
local_embedding = OllamaEmbeddings(model="nomic-embed-text")
#vectorstore = Chroma.from_documents(documents=splits, embedding=local_embedding)


/tmp/ipykernel_1719151/3237731207.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  local_embedding = OllamaEmbeddings(model="nomic-embed-text")


In [11]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=splits, embedding=local_embedding)
# db = FAISS.from_documents(texts, local_embeddings,normalize_L2=True)

In [9]:
from langchain.retrievers.multi_query import MultiQueryRetriever


# Retrieve and generate using the relevant snippets of the blog.
# retriever = vectorstore.as_retriever()
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)


retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .1})
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

/home/uniteomics/miniforge-pypy3/envs/deepseek/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/home/uniteomics/miniforge-pypy3/envs/deepseek/lib/python3.12/site-packages/langchain_core/vectorstores/base.py:1076: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='7e1c0a70-1f6a-4378-899e-db6985fb6698', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simp

"<think>\nOkay, so I need to figure out what Task Decomposition is. From the context given, it's mentioned as a method used in project management. It breaks down larger tasks into smaller, more manageable parts. This makes each task easier to handle and track. By doing this, teams can work on individual pieces without getting overwhelmed by the whole project at once.\n\nI think Task Decomposition is about breaking projects into smaller tasks. That way, each part can be assigned to team members, tracked individually, and then combined later. It probably helps in managing time better and ensuring that everyone knows their role in the project. This process likely improves productivity because it avoids feeling overwhelmed with too much work at once.\n\nI'm not entirely sure if there are different types of decomposition or specific steps involved beyond just breaking tasks down. Maybe it's about identifying dependencies between these smaller tasks as well? Or perhaps using some kind of hie

In [14]:
query="What is Task Decomposition?"
db.similarity_search_with_relevance_scores(query)

/tmp/ipykernel_1719151/3122358776.py:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='18e1bd7a-6830-4af4-be26-94304c56c2c3', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'), -276.9541419998637), (Document(id='c79a423d-9704-42c2-ba69-6b8c0825734b', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-a

[(Document(id='18e1bd7a-6830-4af4-be26-94304c56c2c3', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'),
  -276.9541419998637),
 (Document(id='c79a423d-9704-42c2-ba69-6b8c0825734b', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 11. Illustration of how HuggingGPT works. (Image source: Shen et al.

In [16]:
from langchain.vectorstores import FAISS
# db = FAISS.from_documents(documents=splits, embedding=local_embedding)
db = FAISS.from_documents(documents=splits, embedding=local_embedding,normalize_L2=True)

In [17]:
query="What is Task Decomposition?"
db.similarity_search_with_relevance_scores(query)

[(Document(id='e5229f5d-5f92-42e2-bfb0-a440cb6451e6', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'),
  0.4236241449371496),
 (Document(id='0fd824ee-c77c-420b-b8b5-8dadcb4458cb', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='The AI assistant can parse user input to several tasks: [{"task": task, "

In [18]:
vectorstore = Chroma.from_documents(documents=splits, embedding=local_embedding)

In [19]:
query="What is Task Decomposition?"
vectorstore.similarity_search_with_relevance_scores(query)

/tmp/ipykernel_1719151/3799112155.py:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='7e1c0a70-1f6a-4378-899e-db6985fb6698', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'), -276.95418515823656), (Document(id='e68237fe-10bb-4187-835f-3dc11461d9f3', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-

[(Document(id='7e1c0a70-1f6a-4378-899e-db6985fb6698', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.'),
  -276.95418515823656),
 (Document(id='e68237fe-10bb-4187-835f-3dc11461d9f3', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent On